# COVID 19 Reported Cases and Deaths
## A Local Perspective
All data automatically imported from John Hopkins University at https://github.com/CSSEGISandData/COVID-19

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown
import ipywidgets as wd
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
#data pulled from John Hopkins University at https://github.com/CSSEGISandData/COVID-19
us_confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
us_death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [3]:
#cleaning data and preparing a single sheet for CA
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)
today = us_death_df.columns[-1]
us_confirmed_df = us_confirmed_df.rename(columns = { 'province_state': 'state', 'admin2': 'county'})
us_death_df = us_death_df.rename(columns = {'province_state':'state', 'admin2':'county'})

In [4]:
geo = dict()
remove_state = {'Puerto Rico','Grand Princess', 'Diamond Princess','American Samoa','Guam','Virgin Islands', 'Northern Mariana Islands'}
outof = 'Out of'
indices = list()
for i in range(us_death_df.shape[0]):
    slc = us_death_df.iloc[i:i+1]
    state = str.strip(slc['state'].to_string(index=False))
    county = str.strip(slc['county'].to_string(index=False))
    if state not in remove_state:
        if state not in geo.keys():
            geo[state] = list()
        if not county.startswith(outof):# and county not in remove_county:
            geo[state].append(county)
    if county.startswith(outof):# or county in remove_county:
        indices.append(i)
us_death_df=us_death_df.drop(us_death_df.index[indices])
us_confirmed_df =us_confirmed_df.drop(us_confirmed_df.index[indices])

## See What's Been Happening with your County

In [10]:
#@interact(state=geo.keys(), county='Mobile')
stateW = Dropdown(options = geo.keys())
countyW = Dropdown()

def update_countyW_options(*args): # *args represent zero (case here) or more arguments.
    countyW.options = geo[stateW.value]
countyW.observe(update_countyW_options) # Here is the trick, i.e. update cityW.options based on countryW.value.

@interact(state = stateW, county = countyW)
def plot_cases_for_county(state, county):
    labels =['confirmed', 'dead']
    colors = ['blue', 'red']
    mode_size = [6,8] 
    line_size = [4,5]
    
    state_conf_df = us_confirmed_df[us_confirmed_df['state']==state]
    state_death_df = us_death_df[us_death_df['state']==state]
    start_ndx = 100
    
    df_list = [state_conf_df, state_death_df]
    
    fig = go.Figure();
    fig2 = go.Figure();
    
    fig_list = [fig, fig2]
    #TODO: see if can do a graph with a transpose instead of an elaborate enumeration
    for i, df in enumerate(df_list):
        x_data = np.array(list(df.iloc[:, start_ndx:].columns))
        y_data = np.sum(np.asarray(df[df['county'] == county].iloc[:,start_ndx:]),axis = 0)
        fig_list[i].add_trace(go.Scatter(x = x_data, y = y_data, mode='lines+markers',
                name=labels[i],
                line = dict(color=colors[i], width=line_size[i]),
                connectgaps=True,
                text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
                ));
    case_type = ["Confirmed Cases", "Deaths"]
    for i in range(len(fig_list)):    
        fig_list[i].update_layout(
            title= case_type[i] + " in " + county + " County, " + state,
            xaxis_title='Date',
            yaxis_title="No. of "+ case_type[i],
            margin=dict(l=20, r=20, t=40, b=20),
            paper_bgcolor="lightgrey",
            width = 600,
            );
        fig_list[i].update_yaxes(type="linear")
        fig_list[i].show()

interactive(children=(Dropdown(description='state', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Cali…

# See the most affected counties in your state

In [9]:
@interact
def top_ten(state=geo.keys(), number='10'):
    #now need to sort this somehow
    state_conf = us_confirmed_df[us_confirmed_df['state']==state]
    state_dead = us_death_df[us_death_df['state']==state]
    state_tot = pd.merge(state_dead.rename(columns={today:'dead'}), state_conf.rename(columns={today:'confirmed'}), on ='county')
    state_tot = state_tot[['county', 'confirmed', 'dead']]
    #might call graphing functions from here too
    #TODO: want to drop "Out of (statename)" counties and 'Unassigned'
    return state_tot.sort_values(by = ['confirmed'], ascending = False).head(int(number))

interactive(children=(Dropdown(description='state', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Cali…